In [1]:
import pysolr
import spacy
import requests
import datetime
import json
import dateutil.parser

nlp = spacy.load('en_core_web_lg') #Use the large model, it works the best

### Make a request to the Mapquest service

In [2]:
#MapQuest Geocode service.  More info: https://developer.mapquest.com/documentation/geocoding-api/
mapquest_address_url="https://www.mapquestapi.com/geocoding/v1/address?key=AuqdPFEWYhm7rZRN5hX5HeWSKgaO2u7d&location="
def geocode(text):
    req = requests.get(mapquest_address_url+text)
    jsn = req.json()
    loc = jsn["results"][0]["locations"][0]
    return loc

# Send batches to the MapQuest Geocode API in up to 100 locations at a time
# Saves on API monthly rate (15000 calls per month in the free tier)
#mapquest_batch_url="http://www.mapquestapi.com/geocoding/v1/batch?&inFormat=json&outFormat=json&key=AuqdPFEWYhm7rZRN5hX5HeWSKgaO2u7d"
def geocode_batch(batch):
    mapquest_batch_url="http://www.mapquestapi.com/geocoding/v1/batch?key=AuqdPFEWYhm7rZRN5hX5HeWSKgaO2u7d&maxResults=1&thumbMaps=false"
    request_locations = []
    reverse_lookup = {}
    response = {}
    
    #Create reverse lookup table for the API response
    for movie in batch:
        movie_id = movie["id"]
        location = movie["location"]
        if movie_id not in response:
            response[movie_id] = []
        if location not in reverse_lookup:
            reverse_lookup[location] = []
        reverse_lookup[location].append(movie_id)
        mapquest_batch_url += "&location=" + location
    print(mapquest_batch_url)
    req = requests.get(mapquest_batch_url)
    jsn = req.json()

    # Map the API location results to the movie IDs
    for result in jsn["results"]:
        if "providedLocation" in result and "locations" in result and len(result["locations"]):
            provided = result["providedLocation"]["location"]
            location = result["locations"][0]
            if location["geocodeQuality"] != "COUNTRY" or location["adminArea1"] != "US":
                if provided in reverse_lookup:
                    for movie_id in reverse_lookup[provided]:
                        response[movie_id].append(location)
                else:
                    print("Not found",provided)
                
    return response

def test_batch():
    enriched_movie_ids = geocode_batch([
        {"id":123,"location":"New York"},
        {"id":123,"location":"Manhattan"},
        {"id":123,"location":"Thailand"},
    ])
    enriched_movie_ids

In [3]:
#us-map-with-latitude-longitude-united-states-latitude-longitude.jpg

#Disambiguate to the nearest lat/lng whole numbers:
def near(loc):
    latlng = loc["latLng"]
    return round(latlng["lat"]),round(latlng["lng"])

#Normalize to City|State|Country:
def norm(loc):
    loctypes = {"City":"","State":"","Country":""}
    location = None
    for i in range(8):
        aai = "adminArea"+str(i)
        aat = aai+"Type"
        if aat in loc.keys() and loc[aat] in loctypes.keys():
            loctypes[loc[aat]] = aai
    for typ in loctypes.keys():
        fld = loctypes[typ]
        if fld in loc.keys() and len(loc[fld]):
            if not location:
                location = loc[fld]
            else:
                location += "|" + loc[fld]
    if not location:
        location = "_UNKNOWN_"
    return location.replace(" ", "_")

### Get entities with SpaCy

In [4]:
# Use a basic finite state machine to chunk proper noun GPEs as one location
# This had the best F1 score from our tests in the test_location_entities notebook
def extract_chunked_locations(text):
    doc = nlp(text)
    gpes = []
    for s in doc.sents:
        curr = []
        isgpe = False
        for t in s:
            if (t.pos_ == 'PROPN' and t.ent_type_ == 'GPE') or (isgpe == True and t.text==','):
                isgpe = True
                if t.text!=',':
                    curr.append(t.text)
            elif isgpe == True:
                gpes.append(' '.join(curr))
                curr = []
                isgpe = False
            else:
                isgpe = False
        if(len(curr)):
            gpes.append(' '.join(curr))
            curr = []
            isgpe = False
    return gpes

### Enrich a title with a location using SpaCy and Mapquest

In [14]:
#Iterate through the movies
def rawTmdbMovies(filename):
    return json.load(open(filename))


def writeTmdmMovies(rawMoviesJson, path):
    with open(path, 'w') as f:
        json.dump(rawMoviesJson, f)

def tmdbMovies(filename="../tmdb.json"):
    tmdbMovies = rawTmdbMovies(filename)
    for movieId, tmdbMovie in tmdbMovies.items():
        yield (movieId, tmdbMovie)

def indexableMovies(filename="../tmdb.json"):
    """ Generates TMDB movies, similar to how ES Bulk indexing
        uses a generator to generate bulk index/update actions """
    for movieId, tmdbMovie in tmdbMovies(filename):
        try:
            releaseDate = None
            if 'release_date' in tmdbMovie and len(tmdbMovie['release_date']) > 0:
                releaseDate = tmdbMovie['release_date'] + 'T00:00:00Z'

            yield {'id': movieId,
                   'title': tmdbMovie['title'],
                   'overview': tmdbMovie['overview'],
                   'tagline': tmdbMovie['tagline'],
                   'directors': [director['name'] for director in tmdbMovie['directors']],
                   'cast': [castMember['name'] for castMember in tmdbMovie['cast']],
                   'genres': [genre['name'] for genre in tmdbMovie['genres']],
                   'release_date': releaseDate,
                   'vote_average': tmdbMovie['vote_average'] if 'vote_average' in tmdbMovie else None,
                   'vote_count': int(tmdbMovie['vote_count']) if 'vote_count' in tmdbMovie else None,
                   'location_entities': tmdbMovie['location_entities'] if 'location_entities' in tmdbMovie else [],
                   }
        except KeyError as k: # Ignore any movies missing these attributes
            continue

In [6]:
#For all the movies, lookup the entities in mapquest, and enrich the record for Solr
def enrich_movies():
    total = 0
    batch = []
    movies = {}
    tmdb_spacy = []
    tmdb_enriched = []
    for movie in indexableMovies():
        location_entities = movie["location_entities"]
        if len(location_entities):
            #found a location - add it to the mapquest batch
            movies["id"] = movie
            for entity in location_entities:
                batch.append({"id":movie["id"],"location":entity})
                total +=1
            if (total>=98):
                geocodes = geocode_batch(batch)
                if "location" not in movie:
                    movie["location"] = []
                for movie_id in geocodes.keys():
                    latlon = movie
                    movie["location"].append()
                index_movies(enriched)
                #locations = geocode_batch(batch)
                batch = []
                movies = {}
                total = 0
        movie.pop('location_entities', None)
        tmdb_enriched.append(movie)
        
    writeTmdmMovies(tmdb_enriched,'../tmdb_enriched.json')

In [9]:
#For all the movies, finds all the entities in titles and overviews, and adds them to the respective movie record
def entitize_movies():
    tmdb_spacy = {}
    for movie_id, movie in tmdbMovies(filename="../tmdb.json"):
        title_entities = []
        overview_entities = []
        if "title" in movie and isinstance(movie["title"], str) and len(movie["title"]):
            title_entities = extract_chunked_locations(movie["title"])

        if "overview" in movie and isinstance(movie["overview"], str) and len(movie["overview"]):
            overview_entities = extract_chunked_locations(movie["overview"])

        movie["location_entities"] = title_entities + overview_entities #TODO: dedup this list
        tmdb_spacy[movie_id] = movie
    writeTmdmMovies(tmdb_spacy,'../tmdb_spacy.json')

In [10]:
entitize_movies()

In [16]:
mc=0
lc=0
for movie in indexableMovies(filename='../tmdb_spacy.json'):
    mc+=1
    if "location_entities" in movie and len(movie["location_entities"]):
        lc+=1
        print("-----------------")
        print(movie["title"])
        print(movie["location_entities"])

-----------------
The Brave-Hearted Will Take the Bride
['India', 'India']
-----------------
The Godfather: Part II
['Sicily', 'New York', 'Las Vegas Hollywood', 'Cuba']
-----------------
Tokyo Story
['Tokyo']
-----------------
Woman in the Dunes
['Tokyo']
-----------------
GoodFellas
['Brooklyn']
-----------------
One Flew Over the Cuckoo's Nest
['Cuckoo']
-----------------
Princess Mononoke
['San']
-----------------
Sunset Boulevard
['Hollywood']
-----------------
Doctor Who: The Day of the Doctor
['London']
-----------------
Leon: The Professional
['New York']
-----------------
City of God
['Rio de Janeiro']
-----------------
American History X
['L.A.']
-----------------
Grave of the Fireflies
['Tokyo', 'Japan']
-----------------
Rear Window
['New York']
-----------------
Once Upon a Time in America
['America', 'Manhattan']
-----------------
Ugetsu
['Japan', 'Omizo']
-----------------
Harakiri
['Japan']
-----------------
The Usual Suspects
['L.A.', 'San Pedro']
-----------------
Se7

['City Island', 'Rizzos']
-----------------
Beverly Hills Cop
['Beverly Hills', 'Detroit', 'Beverly Hills']
-----------------
Mr. Morgan's Last Love
['Paris']
-----------------
Lawless
['Franklin County Virginia']
-----------------
The Sweet Hereafter
['Canada']
-----------------
The Crying Game
['Fergus', 'London', 'Fergus']
-----------------
Malena
['Italy']
-----------------
Micmacs
['Bazil', 'Morocco', 'Bazil', 'Bazil']
-----------------
Evil
['Stockholm']
-----------------
To Live and Die in L.A.
['L.A.']
-----------------
American Hustle
['Jersey']
-----------------
Gosford Park
['Gosford Park', 'England']
-----------------
A Love Song for Bobby Long
['New Orleans']
-----------------
Non-Stop
['New York', 'London']
-----------------
Kids
['New York City']
-----------------
The Lost Boys
['California']
-----------------
Spanish Affair
['Seville', 'Andalucia', 'Amaia']
-----------------
Lust, Caution
['Shanghai']
-----------------
Speed
['Los Angeles', 'LA']
-----------------
Aster

Hocus Pocus
['Salem']
-----------------
Machete
['Texas']
-----------------
Unstoppable
['Stanton Pennsylvania']
-----------------
Outbreak
['USA']
-----------------
French Connection II
['New York']
-----------------
Get Shorty
['Hollywood']
-----------------
Demolition Man
['San Angeles', 'Phoenix']
-----------------
Flyboys
['U.S.']
-----------------
Crocodile Dundee
['New York', 'Dundee', 'Dundee', 'Dundee']
-----------------
Special Forces
['Afghanistan']
-----------------
Memphis Belle
['Dresden']
-----------------
Pride and Glory
['New York City']
-----------------
The Wackness
['New York City']
-----------------
Kiss of the Dragon
['Paris']
-----------------
The Blair Witch Project
['Burkittsville Maryland']
-----------------
The Santa Clause
['Santa']
-----------------
The Ramen Girl
['Tokyo']
-----------------
An American Tail: Fievel Goes West
['America']
-----------------
Two Night Stand
['Manhattan']
-----------------
Something's Gotta Give
['Marin', 'Marin']
-------------

Eraser
['U.S.']
-----------------
A Walk in the Woods
['England', 'U.S.']
-----------------
The Ringer
['Knoxville']
-----------------
Waiting for Forever
['Hollywood']
-----------------
Blue Crush
['Hawaii']
-----------------
The Net
['Mexico', 'Mexico', 'U.S.']
-----------------
Alexander
['Persian Empire', 'India']
-----------------
Under Siege 2: Dark Territory
['Washington DC']
-----------------
Wild Hogs
['New Mexico']
-----------------
War, Inc.
['United States']
-----------------
To Rome with Love
['Rome', 'Rome']
-----------------
Scream 3
['Sydney']
-----------------
Les Tuche 2: Le rêve américain
['Los Angeles']
-----------------
Escape from L.A.
['L.A.', 'Los Angeles', 'Los Angeles']
-----------------
The Campaign
['North Carolina', 'United States']
-----------------
The Messengers
['North Dakota']
-----------------
All the King's Men
['Louisiana']
-----------------
The Last Exorcism
['Louisiana']
-----------------
Risen
['Jerusalem']
-----------------
Hostel: Part II
['Slo

['France']
-----------------
Salvador (Puig Antich)
['Salvador']
-----------------
Rain over Santiago
['Santiago']
-----------------
Solo Sunny
['East', 'Berlin']
-----------------
Or (My Treasure)
['Tel Aviv']
-----------------
11'09''01 - September 11
['New York']
-----------------
Twenty Days Without War
['Tashkent', 'Tashkent']
-----------------
Porgy and Bess
['Charleston South Carolina']
-----------------
Exodus
['Israel', 'Israel', 'Cyprus']
-----------------
The Door in the Floor
['East Hampton New York']
-----------------
A Streetcar Named Desire
['New Orleans']
-----------------
Lucky You
['Las Vegas']
-----------------
Manderlay
['Dogville']
-----------------
Fools Rush In
['New York City', 'Las Vegas']
-----------------
The Man You Had in Mind
['Oregon']
-----------------
Jezebel
['New Orleans']
-----------------
Live and Become
['Ethiopia', 'Israel']
-----------------
Golden Door
['Sicily', 'US', 'States', 'New York', 'States', 'Sicily']
-----------------
The Promise
['Kun

...

['Hollywood']
-----------------
Gambling City
['Gambling City']
-----------------
All for the Winner
['China']
-----------------
Beautiful Girl
['Odessa']
-----------------
I, Madman
['Virginia', 'Virginia']
-----------------
Kawa
['New Zealand', 'Websurfer']
-----------------
A Man Called Magnum
['Milan', 'Naples']
-----------------
Occupant
['Manhattan']
-----------------
Salon Kitty
['Kitty', 'Germany']
-----------------
ReGeneration
['America']
-----------------
Ankur
['India']
-----------------
8: The Mormon Proposition
['California']
-----------------
Beverly Hills Chihuahua 3 - Viva La Fiesta!
['Beverly Hills']
-----------------
Naturally Native
['America']
-----------------
The Song Remains the Same
['NYC']
-----------------
Neil Young Journeys
['Toronto', 'Ohio', 'Ontario']
-----------------
Welcome to the Jungle
['New Guinea']
-----------------
Ninette
['France']
-----------------
Kadosh
['Jerusalem']
-----------------
Dr. Crippen
['Canada']
-----------------
Hot Rods To Hell

Stray Dogs
['Afghanistan']
-----------------
The Silence
['Khorshid', 'Tajikestan', 'Khorshid', 'Khorshid']
-----------------
A Moment of Innocence
['Makmahlbaf']
-----------------
Half Moon
['Iran', 'Kurdistan']
-----------------
The Great Texas Dynamite Chase
['Texas']
-----------------
Jeanne Dielman 23, Quai du Commerce 1080 Bruxelles
['Quai du Commerce', 'Bruxelles']
-----------------
New York Cop
['Toshi', 'America', 'New York City']
-----------------
Mr. Denning Drives North
['North']
-----------------
Mr. Arkadin
['Madrid', 'London', 'US', 'Corinth', 'US']
-----------------
Suck
['Canada', 'USA']
-----------------
Bending The Rules
['New Orleans']
-----------------
Lupin the Third: Episode 0: First Contact
['Helmeth']
-----------------
The Dark Mirror
['Hollywood']
-----------------
Bustin' Down the Door
['Hawaii', 'Australia', 'South Africa']
-----------------
The Killing of John Lennon
['Hawaii']
-----------------
Shakespeare Wallah
['India', 'India', 'Bollywood']
-----------

['Persia']
-----------------
The Last Movie
['Peru', 'Kansas']
-----------------
A Yank in the R.A.F.
['Canada', 'New York', 'England']
-----------------
The Miracle of Our Lady of Fatima
['Fatima Portugal']
-----------------
The King of Kings
['Hollywood']
-----------------
The Wild and Wonderful Whites of West Virginia
['West Virginia', 'Boone County West Virginia']
-----------------
The Lawless Frontier
['Sheriff']
-----------------
No Time For Sergeants
['Georgia']
-----------------
Ayn Rand: A Sense of Life
['St. Petersberg Russia', 'America', 'United States']
-----------------
Roads to Koktebel
['Koktebel', 'Moscow', 'Koktebel']
-----------------
Valentin
['late-1960s Buenos Aires']
-----------------
Lost Embrace
['Buenos Aires', 'Buenos Aires']
-----------------
Inside Daisy Clover
['Hollywood']
-----------------
Pokémon the Movie Black: Victini and Reshiram
['Kingdom', 'People', 'Vale', 'Eindoak', 'Iris', 'Eindoak']
-----------------
Master of the Universe
['Germany', 'Frankfur

-----------------
Lilian's Story
['Sydney']
-----------------
They Bite
['Florida']
-----------------
Busses Roar
['California']
-----------------
The Black Power Mixtape 1967-1975
['America']
-----------------
Ten Thousand Saints
['New York City']
-----------------
Son of Babylon
['Iraq']
-----------------
The Blind Sunflowers
['Ourense Spain', 'Ourense']
-----------------
Paint Your Wagon
['Michigan', 'California']
-----------------
Post Mortem
['Chile']
-----------------
PoliWood
['U.S.']
-----------------
S.I.S.
['Los Angeles']
-----------------
Johns
['Santa Monica Blvd', 'L.A.', 'Branson MO']
-----------------
Rockaway
['States', 'Afghanistan', 'Rockaway Queens']
-----------------
Love Likes Coincidences
['Ankara', 'Deniz', 'Ankara', 'Istanbul']
-----------------
Drunk Stoned Brilliant Dead: The Story of the National Lampoon
['Hollywood']
-----------------
Meet Monica Velour
['Indiana', 'Weinermobile']
-----------------
In Football We Trust
['Salt Lake City Utah']
---------------

['Rome', 'Rome']
-----------------
The Last of the High Kings
['Dublin']
-----------------
Hit Lady
['Texas', 'L.A.']
-----------------
Lovely to Look At
['Paris', 'New York']
-----------------
Project Nim
['Nim']
-----------------
Louis Theroux: The Most Hated Family in America
['Louis']
-----------------
New York: A Documentary Film
['New York', 'New York City', 'America']
-----------------
Under the Stars
['Navarre']
-----------------
Reap the Wild Wind
['Mississippi', 'Key West', 'Charleston']
-----------------
The Left Hand of God
['China', 'China']
-----------------
Monday Night Mayhem
['Dallas']
-----------------
Texas Carnival
['Texas', 'Texas']
-----------------
Liliom
['Budapest']
-----------------
Fog City Mavericks
['San Francisco', 'Indiana', 'Cuckoo', 'Incredibles']
-----------------
Take a Girl Like You
['South', 'England']
-----------------
Finding Normal
['Hamptons', 'North Carolina']
-----------------
Douce
['Paris', 'Douce', 'Irène', 'Douce', 'Quebec', 'Irène']
-----

In [17]:
print(lc,mc)

10703 27760


In [9]:
#Solr Client
solr = pysolr.Solr('http://localhost:8983/solr/tmdb')

#Print the Title, Release Date, and Overview from TMDB
def printresults(res):
    for r in res:
        print('\n---')
        print('\n\t'.join([r["title"][0],r["lat"],r["lng"],r["overview"][0]]))

def solrquery(q):
    return "title_en:("+q+")^1.2 overview_en:("+q+")"
        
#Enrich and Search a text query
def search(query):
    enriched = enrich_query_location(query)
    q = solrquery(enriched["q"])
    print("Searching for `" + query + "` ...")
    print("  q = " + q)
    if "fq" in enriched and isinstance(enriched["fq"], str):
        fq = enriched["fq"]
        print("  fq = " + fq)
        res = solr.search(q=q,fq=fq,fl="title,overview,lat,lng",rows=3)
    else:
        print("  ...no enrichments")
        res = solr.search(q=q,fl="title,overview,release_date",rows=3)

    return res

### Let's try it with some good examples

In [1]:
printresults(search("harry potter last 5 years"))

NameError: name 'printresults' is not defined

In [11]:
printresults(search("indiana jones 1/1/1980 to 12/31/1987"))

Searching for `indiana jones 1/1/1980 to 12/31/1987` ...
  q = title_en:(indiana jones )^1.2 overview_en:(indiana jones )
  fq = release_date:[1980-01-01T00:00:00Z TO 1988-01-01T00:00:00Z]

---
Indiana Jones and the Temple of Doom
	1984-05-23T00:00:00Z
	After arriving in India, Indiana Jones is asked by a desperate village to find a mystical stone. He agrees – and stumbles upon a secret cult plotting a terrible plan in the catacombs of an ancient palace.

---
Raiders of the Lost Ark
	1981-06-12T00:00:00Z
	When Dr. Indiana Jones – the tweed-suited professor who just happens to be a celebrated archaeologist – is hired by the government to locate the legendary Ark of the Covenant, he finds himself up against the entire Nazi regime.

---
Guyana Tragedy: The Story of Jim Jones
	1980-04-15T00:00:00Z
	The story of the Peoples Temple cult led by Jim Jones and the events leading up to one of the largest mass suicides in history.


In [12]:
printresults(search("harry potter goblet"))

Searching for `harry potter goblet` ...
  q = title_en:(harry potter goblet)^1.2 overview_en:(harry potter goblet)
  ...no enrichments

---
Harry Potter and the Goblet of Fire
	2005-11-05T00:00:00Z
	Harry starts his fourth year at Hogwarts, competes in the treacherous Triwizard Tournament and faces the evil Lord Voldemort. Ron and Hermione help Harry manage the pressure – but Voldemort lurks, awaiting his chance to destroy Harry and all that he stands for.

---
Harry Potter and the Philosopher's Stone
	2001-11-16T00:00:00Z
	Harry Potter has lived under the stairs at his aunt and uncle's house his whole life. But on his 11th birthday, he learns he's a powerful wizard -- with a place waiting for him at the Hogwarts School of Witchcraft and Wizardry. As he learns to harness his newfound powers with the help of the school's kindly headmaster, Harry uncovers the truth about his parents' deaths -- and about the villain who's to blame.

---
A Very Potter Musical
	2009-04-09T00:00:00Z
	In Apri